In [25]:
from transformers import AutoModel
from transformers import AutoTokenizer
import numpy as np


MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModel.from_pretrained(MODEL)

In [26]:
encoded_input

{'input_ids': tensor([[    0,   118,   657,    47, 23513,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [27]:
text = "i love you 3000"
encoded_input = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    output = model(**encoded_input)

last_hidden_state = output.last_hidden_state  # shape: [1, seq_len, hidden_size]

embedding = last_hidden_state

In [28]:
embedding.shape

torch.Size([1, 6, 768])

In [29]:
embedding = embedding.squeeze()

In [30]:
embedding.shape

torch.Size([6, 768])

In [31]:
tokenizer.convert_ids_to_tokens(encoded_input.input_ids[0])

['<s>', 'i', 'Ġlove', 'Ġyou', 'Ġ3000', '</s>']

### RNN implementation

In [80]:
import torch
import torch.nn as nn

class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.Wax = nn.Parameter(torch.randn(input_size, hidden_size) * 0.01)
        self.Waa = nn.Parameter(torch.randn(hidden_size, hidden_size) * 0.01)
        self.Wya = nn.Parameter(torch.randn(hidden_size, 4) * 0.01)
        self.ba = nn.Parameter(torch.zeros(1, hidden_size))


    def forward(self, inputs):

        a_t = torch.zeros(1, self.hidden_size)  # Initial hidden state
        for x_t in inputs:
            x_t = x_t.unsqueeze(0)  # new Shape (1, input_size)

            a_t = torch.tanh(x_t @ self.Wax + a_t @ self.Waa + self.ba)  # Update hidden state
            # we can compute y_t here but its of no use since we are not going to backprop from each unit unless it is the last one

        y = a_t @ self.Wya
        return y

In [34]:
from datasets import load_dataset

In [88]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/google-research-datasets/poem_sentiment/" + splits["train"])

In [89]:
df.head()

,id,verse_text,label
0,0,with pale blue berries. in these peaceful shad...,1
1,1,"it flows so long as falls the rain,",2
2,2,"and that is why, the lonesome day,",0
3,3,"when i peruse the conquered fame of heroes, an...",3
4,4,of inward strife for truth and liberty.,3


In [90]:
df.shape

(892, 3)

In [91]:
def create_embedding(text):
  temp = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    output = model(**temp)
  return output.last_hidden_state.squeeze()

In [92]:
from tqdm import tqdm

In [96]:
embeddings = []
labels = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    text = row["verse_text"]  # or the actual text column name
    label = row["label"]

    embedding = create_embedding(text)
    embeddings.append((embedding, text))
    labels.append(label)

100%|██████████| 892/892 [01:46<00:00,  8.34it/s]


In [98]:
labels[:2]

[1, 2]

In [99]:
from sklearn.model_selection import train_test_split

# Split into 80% train, 20% test
xtrain, xtest, ytrain, ytest = train_test_split(
    embeddings, labels, test_size=0.2, random_state=42
)

In [101]:
ytrain = torch.tensor(ytrain)
ytest = torch.tensor(ytest)

In [103]:
  for i in range(len(xtrain[:2])):
      x = xtrain[i][0]  # shape: (1, 768)
      y = ytrain[i].unsqueeze(0)  # shape: (1,1)
      print(x.shape, y.shape)

torch.Size([12, 768]) torch.Size([1])
torch.Size([15, 768]) torch.Size([1])


In [104]:
import torch
import torch.nn as nn
import torch.optim as optim


input_size = 768  # size of embedding from BERT
hidden_size = 128
num_classes = 4

rnn = RNNClassifier(input_size=input_size, hidden_size=hidden_size)


loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), lr=0.001)


epochs = 50
for epoch in range(epochs):
    total_loss = 0
    correct = 0

    for i in range(len(xtrain)):
        x = xtrain[i][0]  # shape: (seq_len, 768)
        y = ytrain[i].unsqueeze(0)  # shape: (1,1)

        output = rnn(x)  # shape: (1, 4)
        # print("got output", output)

        loss = loss_fn(output, y)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if torch.argmax(output) == y.item():
            correct += 1

    accuracy = correct / len(xtrain)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch 1/50 - Loss: 533.6700, Accuracy: 0.7181
Epoch 2/50 - Loss: 443.7203, Accuracy: 0.7616
Epoch 3/50 - Loss: 391.7350, Accuracy: 0.7980
Epoch 4/50 - Loss: 354.8313, Accuracy: 0.8107
Epoch 5/50 - Loss: 331.6043, Accuracy: 0.8177
Epoch 6/50 - Loss: 278.0318, Accuracy: 0.8443
Epoch 7/50 - Loss: 247.4763, Accuracy: 0.8766
Epoch 8/50 - Loss: 228.2004, Accuracy: 0.8892
Epoch 9/50 - Loss: 207.8478, Accuracy: 0.8962
Epoch 10/50 - Loss: 189.9188, Accuracy: 0.9144
Epoch 11/50 - Loss: 156.5796, Accuracy: 0.9299
Epoch 12/50 - Loss: 159.2259, Accuracy: 0.9243
Epoch 13/50 - Loss: 137.6615, Accuracy: 0.9355
Epoch 14/50 - Loss: 143.2635, Accuracy: 0.9271
Epoch 15/50 - Loss: 120.5652, Accuracy: 0.9453
Epoch 16/50 - Loss: 118.1064, Accuracy: 0.9467
Epoch 17/50 - Loss: 114.5644, Accuracy: 0.9439
Epoch 18/50 - Loss: 90.6434, Accuracy: 0.9635
Epoch 19/50 - Loss: 67.4000, Accuracy: 0.9762
Epoch 20/50 - Loss: 56.5776, Accuracy: 0.9762
Epoch 21/50 - Loss: 80.8701, Accuracy: 0.9621
Epoch 22/50 - Loss: 46.270

In [105]:
import torch.nn.functional as F

id2label = {0: "negative", 1: "positive", 2: "no impact", 3: "mixed"}  # Edit if needed

def test(rnn, xtest):
    rnn.eval()  # Disable dropout, etc.

    for emb in xtest:
        with torch.no_grad():
            logits = rnn(emb[0])          # (1, 4)
            probs = F.softmax(logits, dim=1)   # Convert logits to probabilities
            pred_class = torch.argmax(probs, dim=1).item()

            print(f"Text: \"{emb[1]}\"")
            print(f"→ Prediction: {id2label[pred_class]} (Confidence: {probs[0, pred_class]:.4f})")
            print("-" * 60)


In [112]:
test(rnn, xtest[5:10])

Text: "you turn, o jeanne, on our mystery"
→ Prediction: no impact (Confidence: 0.9476)
------------------------------------------------------------
Text: "and glance securely round."
→ Prediction: positive (Confidence: 0.6803)
------------------------------------------------------------
Text: "then along the river strand,"
→ Prediction: no impact (Confidence: 1.0000)
------------------------------------------------------------
Text: "flood his black hearthstone till its flames expire,"
→ Prediction: negative (Confidence: 0.9843)
------------------------------------------------------------
Text: "a million torches lighted by thy hand"
→ Prediction: positive (Confidence: 0.6276)
------------------------------------------------------------
